<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [3]:
import os
import time
import librosa
import numpy as np
import pandas as pd
from keras.models import load_model

In [2]:
model1 = load_model('./data/baseline_clean_zeros1.pkl')

background_0001.wav


In [5]:
df = pd.read_csv('./data/data_v_7_stc/meta/fold1_test.txt', sep='\t')
df.columns = ['filename', 'xz1', 'xz2', 'time', 'target']
targets = list(df['target'].unique())
targets.sort()

In [6]:
def score_filename(filename):
    
    data, _ = librosa.load(filename, sr=8000)
    
    samples = []
    for i in range(0, data.shape[0], 8000):
        sub_sample = data[i: i + 8000]
        if sub_sample.shape[0] < 8000:
            sub_sample = np.array(list(sub_sample) + [0 for _ in range(8000 - sub_sample.shape[0])])
        samples.append(sub_sample)

    x = np.vstack(samples)
    x = np.expand_dims(x, axis=2)
    
    y = model1.predict(x)
    
    y = y.mean(axis=0)
    
    y = np.exp(y) / np.exp(y).sum()
    return y

In [8]:
ff = open('./output.txt', 'w')

for f in os.listdir('/Volumes/NO NAME/data_v_7_stc_test'):
    filename = '/Volumes/NO NAME/data_v_7_stc_test/' + f
    score = score_filename(filename)
    
    target_ind = score.argmax()
    target = targets[target_ind]
    proba = score[target_ind]
    
    ff.write(f + '\t' + str(proba) + '\t' + target + '\n')

ff.close()

In [58]:
predicted_targets = []
predicted_probs = []

t = time.time()
for i, row in df.iterrows():
    
    if i % 100 == 0:
        print(i, time.time() - t)
        t = time.time()
    
    y = score_filename('./data/data_v_7_stc/audio/' + row['filename'])
    target_ind = y.argmax()
    target = targets[target_ind]
    prob = y[target_ind]
    predicted_targets.append(target)
    predicted_probs.append(prob)

0 0.0009658336639404297
100 18.446014881134033
200 15.590487718582153
300 24.505233764648438
400 5.439617872238159
500 10.356916189193726
600 7.6485912799835205
700 5.70461106300354
800 4.363824844360352
900 2.9724698066711426
1000 3.2124979496002197
1100 7.435490131378174
1200 12.942426919937134
1300 14.056018829345703
1400 16.58789086341858
1500 10.579710006713867
1600 10.093451023101807
1700 7.218870162963867
1800 22.56569790840149
1900 46.14298987388611
2000 22.945905923843384
2100 7.443116903305054
2200 8.504034757614136


In [60]:
df['pred_target'] = predicted_targets
df['pred_prob'] = predicted_probs

In [62]:
df.shape

(2265, 7)

In [63]:
df[df['target'] != df['pred_target']].shape

(280, 7)

In [64]:
(2265. - 280.) / 2265

0.8763796909492274

In [67]:
df[df['target'] != df['pred_target']].head(5)

,filename,xz1,xz2,time,target,pred_target,pred_prob
47,background_0014_time_stretch_13.wav,office,0.0,2.110975,background,knocking_door,0.159780
184,background_0060_time_stretch_7.wav,office,0.0,4.667075,background,keyboard,0.159167
217,background_0074_time_stretch_13.wav,office,0.0,2.127710,background,keyboard,0.155887
218,background_0074_time_stretch_4.wav,office,0.0,3.133560,background,speech,0.146438
219,background_0074_time_stretch_5.wav,office,0.0,2.983537,background,speech,0.153166


In [70]:
df_conf = df.groupby(['target', 'pred_target']).size().reset_index()

In [74]:
df_conf.pivot('target', 'pred_target').fillna(0)

0                                                     \
pred_target   background   bags   door keyboard knocking_door   ring speech   
target                                                                        
background         220.0    0.0    0.0      2.0           1.0    0.0    2.0   
bags                 1.0  243.0    1.0      3.0           0.0    0.0    0.0   
door                69.0   13.0  505.0     42.0          34.0    4.0    9.0   
keyboard            14.0    0.0    3.0    226.0           0.0    1.0    1.0   
knocking_door       11.0    0.0    1.0      3.0         306.0    0.0   11.0   
ring                 6.0    0.0    1.0      0.0           0.0  133.0    1.0   
speech               0.0    0.0    0.0      0.0           0.0    1.0   55.0   
tool                 3.0   20.0    4.0      0.0           0.0    8.0    0.0   

                      
pred_target     tool  
target                
background       0.0  
bags             0.0  
door             8.0  
keyboard         0.0  
knocking_door    0.0  
ring             2.0  
speech           0.0  
tool           297.0